In [30]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/MyDrive/face3d/face3d/mesh/cython

/content/gdrive/MyDrive/face3d/face3d/mesh/cython


In [3]:
!python setup.py build_ext --inplace


running build_ext
skipping 'mesh_core_cython.cpp' Cython extension (up-to-date)


In [4]:
!python setup.py install

running install
running build
running build_ext
skipping 'mesh_core_cython.cpp' Cython extension (up-to-date)
running install_lib
copying build/lib.linux-x86_64-3.8/mesh_core_cython.cpython-38-x86_64-linux-gnu.so -> /usr/local/lib/python3.8/dist-packages
running install_egg_info
Writing /usr/local/lib/python3.8/dist-packages/mesh_core_cython-0.0.0.egg-info


In [5]:
import sys
sys.path.append('/content/gdrive/MyDrive/face3d/')
import face3d
from face3d import mesh
from face3d.morphable_model import MorphabelModel

In [6]:
import cv2
import dlib
import numpy as np
from skimage import io
import os

Load pictures, initialize face detection and key point detection

In [38]:
im = cv2.imread('/content/gdrive/MyDrive/face3d/examples/Data/wu.jpg')
print(im.shape)
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
h, w, c = im.shape
# Face detection
detector = dlib.get_frontal_face_detector()
# 68 Key point detection
predictor = dlib.shape_predictor("/content/gdrive/MyDrive/face3d/examples/Data/shape_predictor_68_face_landmarks.dat")

(472, 700, 3)


Conduct face detection and draw key points

In [39]:
rects = detector(im, 1)
for k, d in enumerate(rects):
    print("dets{}".format(d))
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
 
    # Use predictor to recognize face key points. Shape is the returned result
    shape = predictor(im, d)
    # Obtain the coordinates of the first and second points (relative to the picture rather than the framed face)
    print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))

    for index, pt in enumerate(shape.parts()):
        print('Part {}: {}'.format(index, pt))
        pt_pos = (pt.x, pt.y)
        cv2.circle(im, pt_pos, 1, (255, 0, 0), 2)
        # Use cv2.putText to output 1-68
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(im, str(index + 1), pt_pos, font, 0.3, (0, 0, 255), 1, cv2.LINE_AA)


# Key points (68)
shape = predictor(im, rects[0])
rects = [(rects[0].tl_corner().x, rects[0].tl_corner().y), 
         (rects[0].br_corner().x, rects[0].br_corner().y)]
landmarks = np.zeros((68, 2))
# Draw key points
for i, p in enumerate(shape.parts()):
    landmarks[i] = [p.x, p.y]

dets[(277, 63) (598, 384)]
Detection 0: Left: 277 Top: 63 Right: 598 Bottom: 384
Part 0: (282, 138), Part 1: (287, 183) ...
Part 0: (282, 138)
Part 1: (287, 183)
Part 2: (296, 226)
Part 3: (306, 269)
Part 4: (318, 309)
Part 5: (338, 344)
Part 6: (368, 373)
Part 7: (402, 395)
Part 8: (443, 403)
Part 9: (484, 394)
Part 10: (521, 370)
Part 11: (554, 341)
Part 12: (578, 306)
Part 13: (591, 263)
Part 14: (598, 219)
Part 15: (605, 172)
Part 16: (609, 125)
Part 17: (296, 106)
Part 18: (314, 87)
Part 19: (343, 85)
Part 20: (371, 92)
Part 21: (398, 104)
Part 22: (447, 101)
Part 23: (476, 89)
Part 24: (508, 82)
Part 25: (540, 84)
Part 26: (565, 102)
Part 27: (423, 138)
Part 28: (423, 169)
Part 29: (423, 199)
Part 30: (422, 230)
Part 31: (394, 250)
Part 32: (409, 255)
Part 33: (426, 258)
Part 34: (443, 254)
Part 35: (460, 249)
Part 36: (327, 140)
Part 37: (344, 129)
Part 38: (366, 131)
Part 39: (385, 147)
Part 40: (364, 151)
Part 41: (343, 151)
Part 42: (471, 145)
Part 43: (489, 128)
Part 44: (51

In [17]:
print(shape.parts)

<bound method PyCapsule.parts of <_dlib_pybind11.full_object_detection object at 0x7f85cbbccd70>>


In [40]:
# Transform 2D coordinate points landmarks into 3D coordinate system
def from_image(vertices, h, w):
    ''' change vertices to image coord system
    3d system: XYZ, center(0, 0, 0)
    2d image: x(u), y(v). center(w/2, h/2), flip y-axis.
    Args:
        vertices: [nver, 3]
        h: height of the rendering
        w : width of the rendering
    Returns:
        projected_vertices: [nver, 3]
    '''
    image_vertices = vertices.copy()

    # move to center of image
    image_vertices[:,0] = image_vertices[:,0] - w/2
    # flip vertices along y-axis.
    image_vertices[:,1] = -(- h + image_vertices[:,1] + 1)
    image_vertices[:, 1] = image_vertices[:, 1] - h / 2
    return image_vertices

Load BFM model

In [41]:
# load bfm model
bfm = MorphabelModel('/content/gdrive/MyDrive/face3d/examples/Data/BFM/Out/BFM.mat')
print('init bfm model success')
# Coordinate system transformation
x = from_image(landmarks, h, w)
X_ind = bfm.kpt_ind

init bfm model success


3D parameter fitting

In [42]:
# fit. The 2d coordinates of the incoming key points and the index of the corresponding vertices in the model
fitted_sp, fitted_ep, fitted_s, fitted_angles, fitted_t = bfm.fit(x, X_ind, max_iter = 200)
# The coefficient of texture is directly random
colors = bfm.generate_colors(np.random.rand(bfm.n_tex_para, 1))
colors = np.minimum(np.maximum(colors, 0), 1)

In [21]:
print(fitted_sp.shape)

(199, 1)


Through 3DMM definition, shape and expression are calculated according to the fitting coefficient

In [43]:
fitted_vertices = bfm.generate_vertices(fitted_sp, fitted_ep)

In [44]:
tp = bfm.get_tex_para('random')
print(tp.shape)
colors = bfm.generate_colors(tp)
print(colors.shape)
colors = np.minimum(np.maximum(colors, 0), 1)
print(colors.shape)

(1, 1)
(53215, 3)
(53215, 3)


Transform vertices accordingly

In [45]:
# Including translation, rotation and scaling
transformed_vertices = bfm.transform(fitted_vertices, fitted_s, fitted_angles, fitted_t)
print(transformed_vertices.shape)

(53215, 3)


Project 3D model to 2D plane (orthographic projection)

In [46]:
image_vertices = mesh.transform.to_image(transformed_vertices, h, w)

Rendering

In [47]:
fitted_image = mesh.render.render_colors(image_vertices, bfm.full_triangles, colors, h, w)

In [17]:
print(bfm.triangles.shape)

(105840, 3)


Save file

In [48]:
save_folder = '/content/gdrive/MyDrive/face3d/results'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

io.imsave('{}/generated.jpg'.format(save_folder), im)
io.imsave('{}/fitted.jpg'.format(save_folder), fitted_image)

Save the obj model by combining the color of 3d-fitting-pytoch fitting and the vertex of face3d fitting

In [19]:
from scipy.io import loadmat, savemat

In [32]:
def save_obj(path, v, f, c):
    with open(path, 'w') as file:
        for i in range(len(v)):
            file.write('v %f %f %f %f %f %f\n' %
                       (v[i, 0], v[i, 1], v[i, 2], c[i, 0], c[i, 1], c[i, 2]))

        file.write('\n')

        for i in range(len(f)):
            file.write('f %d %d %d\n' % (f[i, 0], f[i, 1], f[i, 2]))

    file.close()
obj_path = save_folder + '/child.obj'

In [49]:
color = loadmat('/content/gdrive/MyDrive/face3d/examples/Data/BFM/Out/color.mat')
color = color['color']

In [50]:
save_obj(obj_path, transformed_vertices, bfm.triangles+1, color)